In [ ]:
from Dataload import dataload

from module import transformer,video_model,invres

import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils
import os
from constant import EMOTIPATH
import time

#Batchfy
#Counts number of correct label
def num_correct(prediction,labels):
    correct=0
    for i,(pred_label,label) in enumerate(zip(prediction,labels)):
        if (pred_label.item()==label.item()):
            correct +=1
    return correct

Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video=os.path.join(EMOTIPATH,"Train")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video=os.path.join(EMOTIPATH,"Val")


train_data=dataload.Video_Frame_Data(Train_label,base_path_v=Train_video,strict_num=26)
valid_data=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=26)

#train_data=dataload.Video_Embedding_Data("train.csv",Train_label)
#valid_data=dataload.Video_Embedding_Data("valid.csv",Val_labels)
            
train_dataloader = DataLoader(train_data, batch_size=32
                       , num_workers=0,shuffle=True)
#valid_data=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=26)

valid_dataloader = DataLoader(valid_data, batch_size=32
                       , num_workers=0)



In [2]:
len(train_data)

2661

In [3]:
train_data[0][0].shape

torch.Size([24, 3, 256, 256])

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#torch.set_default_tensor_type(torch.cuda.FloatTensor)
model=video_model.Video_Feature(joint=False,grayscale=False,classfication=True,frame=24,skip_process=False)
for name, child in model.named_children():
   if not name in ['densenet']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False
model=model.to(device)  
#model=model.double().to(device)

criterion = nn.CrossEntropyLoss()  
loss_func = torch.nn.MSELoss()

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0002, betas=(0.5, 0.999))


densenet is frozen
fc2 is unfrozen
fc3 is unfrozen
posencoding is unfrozen
transformer is unfrozen


In [5]:
output=model(train_data[60][0].unsqueeze(0).to(device))
#loss=criterion(output,(train_data[60][1].unsqueeze(0).to(device)))


In [6]:
output

tensor([[0.3496, 0.3252, 0.3252]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [ ]:
import time

In [ ]:
start = time.time()
print("hello")
end = time.time()
print(end - start)

In [ ]:
def train(num_epochs):
    f = open("data.txt",'a')
    start = time.time()
    #Triaining
    train_loss=[]
    valid_accuracy=[]
    num_epochs=50

    model.train()
    for epochs in range(0,num_epochs):
        first=True
        model.train()
        correct=0
        total_samples=0
        avg_tloss=0
        print("Training Epoch: ", epochs+1,"\n")
        for i_batch, (sample_batched,label) in enumerate(train_dataloader):

            batch_size=sample_batched.size(0)
            optimizer.zero_grad()
            sample_batched=sample_batched.to(device)
            output=model(sample_batched)
            loss=criterion(output,label.to(device))
            #loss=loss_func(output,label.float().to(device))
            loss.backward()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size
            accuracy=correct/(total_samples)
            optimizer.step()
            true_label=label.detach().cpu()
            avg_tloss+=loss.item()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
            if i_batch%20==0:
               # print(label)
                print("Batch: ",i_batch+1,"/",len(train_dataloader))
                print("Batch Recognition loss: ", loss.item())

        print(conf_mat)
        avg_tloss=avg_tloss/len(train_dataloader)
        avg_taccuracy=correct/total_samples
        print("Average_Loss: ",avg_tloss)
        print("Average_Accuracy: ",avg_taccuracy)

        torch.save(model,"pre_embedded_model.h")
        print("Validation\n")

        model.eval()   
        correct=0
        total_samples=0
        avg_vloss=0
        first=True
        for i_batch, (sample_batched,label) in enumerate(valid_dataloader):
            batch_size=sample_batched.size(0)
            sample_batched=sample_batched.to(device)
            output=model(sample_batched)
            loss=criterion(output,label.to(device))
          #  loss=loss_func(output,label.float().to(device))
            avg_vloss+=loss.item()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size

            true_label=label.detach().cpu()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
        print(conf_mat)
        avg_vloss=avg_vloss/len(valid_dataloader)
        print(avg_vloss)
        avg_vaccuracy=correct/(total_samples)
        print("Accuracy: ", avg_vaccuracy)

        data = " %d,%d,%d,%d \n" % (avg_tloss,avg_taccuracy,avg_vloss,avg_vaccuracy)
        f.write(data)

    f.close()


In [ ]:
train(50)

In [ ]:
    model.eval()   
    correct=0
    total_samples=0
    avg_vloss=0
    first=True
    for i_batch, (sample_batched,label) in enumerate(valid_dataloader):
        batch_size=sample_batched.size(0)
        sample_batched=sample_batched.to(device)
        output=model(sample_batched)
        loss=criterion(output,label.to(device))
      #  loss=loss_func(output,label.float().to(device))
        avg_vloss+=loss.item()
        predicted = torch.max(output, 1)
        prediction=predicted.indices.detach().cpu()
        correct +=num_correct(prediction,label)
        total_samples+=batch_size
        
        true_label=label.detach().cpu()
        if first:
            first=False
            conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
        else:
            conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
    print(conf_mat)
    avg_vloss=avg_vloss/len(valid_dataloader)
    print(avg_vloss)
    avg_vaccuracy=correct/(total_samples)
    print("Accuracy: ", avg_vaccuracy)
    
   # data = " %d,%d,%d,%d \n" % (avg_tloss,avg_taccuracy,avg_vloss,avg_vaccuracy)


In [ ]:
valid_data1=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=26)

valid_dataloader1 = DataLoader(valid_data1, batch_size=32
                       , num_workers=0)

In [ ]:
    model.eval()   
    correct=0
    total_samples=0
    avg_vloss=0
    first=True
    model=model.float()
    for i_batch, (sample_batched,_,label) in enumerate(valid_dataloader1):
        batch_size=sample_batched.size(0)
        sample_batched=sample_batched.to(device)
        sample_batched=model.posencoding(model.stack_frame(sample_batched))
        break

In [ ]:
sample_batched

In [ ]:
for i_batch, (sample_batched,label) in enumerate(valid_dataloader):
    debug=sample_batched
    break

In [ ]:
debug

In [ ]:

        output=model(sample_batched)
                                         
        loss=criterion(output,label.to(device))
      #  loss=loss_func(output,label.float().to(device))
        avg_vloss+=loss.item()
        predicted = torch.max(output, 1)
        prediction=predicted.indices.detach().cpu()
        correct +=num_correct(prediction,label)
        total_samples+=batch_size
        
        true_label=label.detach().cpu()
        if first:
            first=False
            conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
        else:
            conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
    print(conf_mat)
    avg_vloss=avg_vloss/len(valid_dataloader)
    print(avg_vloss)
    avg_vaccuracy=correct/(total_samples)
    print("Accuracy: ", avg_vaccuracy)

In [7]:
def train(num_epochs):
#f = open("result.txt",'a')
    start = time.time()
    #Triaining
    train_loss=[]
    valid_accuracy=[]

    model.train()
    for epochs in range(0,1):
        first=True
        model.train()
        correct=0
        total_samples=0
        avg_tloss=0
        print("Training Epoch: ", epochs+1,"\n")
        for i_batch, (sample_batched,_,label) in enumerate(train_dataloader):

            batch_size=sample_batched.size(0)
            optimizer.zero_grad()
            sample_batched=sample_batched.to(device)
            output=model(sample_batched)
            loss=criterion(output,label.to(device))
            #loss=loss_func(output,label.float().to(device))
            loss.backward()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size
            accuracy=correct/(total_samples)
            optimizer.step()
            label=label.cpu()
            avg_tloss+=loss.item()
            if first:
                first=False
                conf_mat=confusion_matrix( label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(label,prediction,labels=[0,1,2])
            if i_batch%20==0:
                #print(label)
                print("Batch: ",i_batch+1,"/",len(train_dataloader))
                print("Batch Recognition loss: ", loss.item())

        print(conf_mat)
        avg_tloss=avg_tloss/len(train_dataloader)
        avg_taccuracy=correct/total_samples
        print("Average_Loss: ",avg_tloss)
        print("Average_Accuracy: ",avg_taccuracy)
    end=time.time()
    
    print(end - start)

In [8]:
train(1)

Training Epoch:  1 

Batch:  1 / 84
Batch Recognition loss:  1.1029760837554932
Batch:  21 / 84
Batch Recognition loss:  0.9834776520729065
Batch:  41 / 84
Batch Recognition loss:  1.006861686706543
Batch:  61 / 84
Batch Recognition loss:  1.0549951791763306
Batch:  81 / 84
Batch Recognition loss:  1.0818843841552734
[[137 391 278]
 [ 77 614 232]
 [ 72 218 642]]
Average_Loss:  1.0028007952939897
Average_Accuracy:  0.5234874107478391
2053.7786338329315
